Dans ce script, nous allons tester différents modèles de type convolutionnelsur le dataset 

résultat:
* l'utilisation des batchs rend l'inférence globale plus rapide
* un modèle a besoin de beaucoup de temps / parametres pour arriver à un certain niveau d'accuracy

In [1]:
import retinoto_py as fovea
args = fovea.Params(do_mask=False)
args

Welcome on macOS-26.1-arm64-arm-64bit-Mach-O	 user laurent	Running on MPS device (Apple Silicon/MacOS)	 - macos_version = 26.1	 with device mps, pytorch==2.9.1
Random seed 1998 has been set.


Params(image_size=224, do_mask=False, do_fovea=False, rs_min=0.0, rs_max=-5.0, padding_mode='zeros', seed=1998, batch_size=64, num_workers=4, in_memory=True, model_name='resnet101', do_scratch=False, num_epochs=10, n_train_stop=32768, n_val_stop=4096, lr=0.001, delta1=0.05, delta2=0.0, weight_decay=0.003, label_smoothing=0.0001, shuffle=True, verbose=True)

# testing a ResNet50 model on the validation dataset

In [2]:
VAL_DATA_DIR = args.DATAROOT / 'Imagenet_full' / 'val'

In [3]:
# val_dataset = fovea.get_dataset(args, VAL_DATA_DIR, n_stop=args.n_val_stop)
# val_loader = fovea.get_loader(args, val_dataset)

In [4]:
model = fovea.load_model(args)
model.eval()
# --- Parameter Counts ---
param_stats = fovea.count_parameters(model)
print("🔢 Parameter Count:")
print(f"  Total:     {param_stats['total_parameters']:,}")
print(f"  Trainable: {param_stats['trainable_parameters']:,}")
print("-" * 50)

# --- Layer Counts ---
print("🧱 Layer Count:")
# 1. Total number of nn.Module objects
total_layers = fovea.count_layers(model)
print(f"  Total Modules (nn.Module): {total_layers}")

# 2. Counting specific layer types
from torch import nn

conv_layers = fovea.count_layers(model, layer_type=nn.Conv2d)
linear_layers = fovea.count_layers(model, layer_type=nn.Linear)
print(f"  Convolutional (nn.Conv2d): {conv_layers}")
print(f"  Linear (nn.Linear): {linear_layers}")
print("=" * 50)

🔢 Parameter Count:
  Total:     44,549,160
  Trainable: 44,549,160
--------------------------------------------------
🧱 Layer Count:
  Total Modules (nn.Module): 287
  Convolutional (nn.Conv2d): 104
  Linear (nn.Linear): 1


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import torchvision
import torch
import numpy as np
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
from torchvision import datasets
from tqdm.auto import tqdm
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
def torch_loader(path: str) -> torch.Tensor:
    """
    Load an image file using torchvision.io.read_image.
    The returned tensor is float32 in the range [0, 1] and has shape (C, H, W).

    Parameters
    ----------
    path : str
        Full path to the image file.

    Returns
    -------
    torch.Tensor
        Float tensor ready for torchvision transforms.
    """
    # read_image returns a UInt8 tensor (C, H, W) with values 0‑255
    img = read_image(path)                # still on CPU
    # Convert to float and normalise
    img = img.float() / 255.0

    # -----------------------------------------------------------------
    # 2‑channel / 1‑channel handling (most transforms expect 3 channels)
    # -----------------------------------------------------------------
    if img.shape[0] == 1:                 # grayscale → replicate to 3 channels
        img = img.repeat(3, 1, 1)
    elif img.shape[0] == 4:               # RGBA → drop alpha (or you could blend)
        img = img[:3, :, :]                # keep only RGB
    elif img.shape[0] not in (3,):
        raise RuntimeError(f"Unsupported number of channels ({img.shape[0]}) in {path}")

    return img

class InMemoryImageDataset(Dataset):
    """Load entire ImageFolder dataset into memory"""
    def __init__(self, root, transform=None, n_stop=0, is_valid_file=None):
        # Use ImageFolder to handle directory structure and class mapping
        image_folder = datasets.ImageFolder(root=root, 
                                            loader=torch_loader,
                                            is_valid_file=is_valid_file,
                                            transform=None, 
                                            )
        
        self.class_to_idx = image_folder.class_to_idx
        self.idx_to_class = {v: k for k, v in self.class_to_idx.items()}
        self.classes = image_folder.classes
        self.transform = transform
        
        # Load all images into memory
        # print("Loading dataset into memory...")
        self.images = []
        self.labels = []
        if n_stop==0:
            n_total = len(image_folder)
            idxs = np.arange(n_total) 
        else:
            n_total = min((n_stop, len(image_folder)))
            idxs = np.random.permutation(len(image_folder))[:n_total].astype(int)

        for idx in tqdm(idxs, desc='Putting images in memory', total=n_total, leave=False):
            self.images.append(image_folder[idx][0])
            self.labels.append(image_folder[idx][1])

        # print(f"Loaded {len(self.images)} images into memory")
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img = self.images[idx]
        label = self.labels[idx]
        
        if self.transform:
            img = self.transform(img)
        
        return img, label


dataset = InMemoryImageDataset(root=VAL_DATA_DIR, transform=None, n_stop=args.n_val_stop)

Putting images in memory:   0%|          | 0/4096 [00:00<?, ?it/s]

In [6]:
dataset

In [7]:
len(dataset.class_to_idx)

1000

In [8]:
dataset.images[0].shape

torch.Size([3, 441, 500])

In [9]:
json_filename = args.data_cache / '11_model_accuracy.json'

# %rm -f {json_filename}  # FORCING RECOMPUTE

if json_filename.exists():
    results = fovea.pd.read_json(json_filename)
else:
    all_results_list = []
    for dataset in ['full']:# <HACK until bbox is finished> fovea.all_datasets:
        VAL_DATA_DIR = args.DATAROOT / f'Imagenet_{dataset}' / 'val'
        val_dataset = fovea.get_dataset(args, VAL_DATA_DIR, n_stop=args.n_val_stop)
        val_loader = fovea.get_loader(args, val_dataset)
        accuracy = fovea.get_validation_accuracy(args, model, val_loader, f"Evaluating {args.model_name} on dataset: {dataset}")
        all_results_list.append({'model_name':args.model_name, 'dataset':dataset, 'accuracy': accuracy})

    results = fovea.pd.DataFrame(all_results_list)
    results['accuracy_str'] = results['accuracy'].apply(lambda x: f"Accuracy: {x * 100:.1f}%")
    results.to_json(json_filename, orient='records', indent=2)

print(f"Evaluation complete.")

Putting images in memory:   0%|          | 0/4096 [00:00<?, ?it/s]

Evaluating resnet101 on dataset: full:   0%|          | 0/64 [00:00<?, ?it/s]

Evaluation complete.


In [10]:
results

,model_name,dataset,accuracy,accuracy_str
0,resnet101,full,0.820312,Accuracy: 82.0%


In [11]:
for _, row in results.iterrows():
    accuracy_percent = row['accuracy'] * 100
    print(f"Accuracy for {row['dataset']}: {accuracy_percent:.3f}%")

Accuracy for full: 82.031%


In [12]:
%cat {json_filename}

[
  {
    "model_name":"resnet101",
    "dataset":"full",
    "accuracy":0.8203125,
    "accuracy_str":"Accuracy: 82.0%"
  }
]

In [13]:
print(results[['dataset', 'accuracy_str']].to_string(index=False))

dataset    accuracy_str
   full Accuracy: 82.0%
